In [ ]:
import pandas as pd
import numpy as np
import os
import plotly.graph_objs as go
from plotly.subplots import make_subplots



In [ ]:
nbpath     = '/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN'
voxel_path = '/home/mgolub4/DLproj/MLTO_2024/0_data/voxel_arrays_npy_by_partno'

dbpath = os.path.join(nbpath, 'dyn_data', 'dyn_stat_database_PINN_ready.csv')


dyndb = pd.read_csv(dbpath)

stress_ser_path = os.path.join(nbpath, 'dyn_data/stress_series_data')

csvs = [p for p in sorted(os.listdir(stress_ser_path)) if p.endswith('untrunc.csv')]


In [ ]:
constit_eqn_params = ['A_opt',
       'B_opt', 'C_opt', 'm_opt', 'n_opt',]

In [ ]:
np.asarray(dyndb[constit_eqn_params].iloc[0])

In [ ]:
params = ['volFrac', 
        'CH_11 scaled', 'CH_22 scaled', 'CH_33 scaled', 'CH_44 scaled', 'CH_55 scaled', 'CH_66 scaled',
        'CH_12 scaled', 'CH_13 scaled','CH_23 scaled',
        'EH_11 scaled', 'EH_22 scaled', 'EH_33 scaled',
        'GH_23 scaled', 'GH_13 scaled', 'GH_12 scaled', 
        'vH_12 scaled', 'vH_13 scaled', 'vH_23 scaled', 'vH_21 scaled', 'vH_31 scaled','vH_32 scaled',
        'KH_11 scaled', 'KH_22 scaled', 'KH_33 scaled', 
        'kappaH_11 scaled', 'kappaH_22 scaled', 'kappaH_33 scaled']

In [ ]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import os


In [ ]:

class PINN_Dataset(Dataset):
    def __init__(self, params, split_dataframe,
                 feat_vec_directory='/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/voxel_embedding_feature_maps', 
                 stress_series_directory='/home/mgolub4/DLproj/MLTO_2024/3_Dynamic_PINN_RNN/dyn_data/stress_series_data', 
                 stress_ser_suffix = '_proct_gaus_btrlp_fftlp',
                 predicted_parameters=True,
                  ):
        self.df = split_dataframe
        self.featvec_dir = feat_vec_directory # for pulling the feature vectors
        self.stress_ser_dir = stress_series_directory # for pulling the time series files
        self.params = params
        self.predicted_parameters = predicted_parameters
        self.const_eqn_params = ['A_opt', 'B_opt', 'C_opt', 'm_opt', 'n_opt',]
        self.stress_ser_suffix = stress_ser_suffix


    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):

        dyn_series_fname = self.df['dyn_file_name_original'].iloc[idx]

        sig_pl = self.df[self.df['dyn_file_name_original'] == dyn_series_fname]['plateau_stress_g'].values[0]
        W = self.df[self.df['dyn_file_name_original'] == dyn_series_fname]['energy_absorbed_g'].values[0]

        # feature vector from convolutional neural network convolutional layers output
        featvec_fname = self.df['conv_feat_vec'].iloc[idx] + '.npy'
        featvec_path = os.path.join(self.featvec_dir, featvec_fname)
        featvec = np.load(featvec_path)

        # constitutive equation parameters
        constit_eqn_coeffs = np.asarray(self.df[self.const_eqn_params].iloc[idx])

        # predicted parameters
        if self.predicted_parameters:
            paramvec = np.asarray([self.df[f'pred {par}'].iloc[idx] for par in self.params])
        else:
            paramvec = np.asarray([self.df[f'{par}'].iloc[idx] for par in self.params])

        # stress_series -- for now (April 24), Imma use the truncated datasets, because I think padded batches for RNNs in pytorch will take care of differing lengths
        stress_ser_fname = dyn_series_fname + self.stress_ser_suffix
        stress_ser_path = os.path.join(self.stress_ser_dir, stress_ser_fname+'.csv')
        stress_series = np.asarray(pd.read_csv(stress_ser_path)['stress_bottom_gsreg'])
        strain = np.asarray(pd.read_csv(stress_ser_path)['Strain'])
                   
        return featvec, paramvec, stress_series, constit_eqn_coeffs, W, sig_pl, strain

In [ ]:
from sklearn.model_selection import train_test_split as TTS

In [ ]:
idxTr, idxRem = TTS(dyndb, stratify = dyndb['topology_family'], random_state=42, train_size = 0.8)
idxVal, idxTe = TTS(idxRem, random_state = 42, test_size=0.5)

In [ ]:
dat = PINN_Dataset(params, idxTr)

In [ ]:
next(iter(dat))[6].shape


In [ ]:
def PINN_loss(dynamic_stress_PINN, data, constit_eqn_coeffs:tuple, offset=-0.01):

    A_pred, B_pred, C_pred, m_pred, n_pred = constit_eqn_coeffs

    strain = data[6]

    stress_pred = dynamic_stress_PINN[0]

    stress_series_constit_eqn = A_pred * (strain + offset)**m_pred + B_pred*((strain + offset)/(C_pred-(strain + offset)))**n_pred

    sig_pl_pred = np.mean(stress_series_constit_eqn[200:400])
    sig_pl_data = data[5]

    stress_start = -1*offset+1e3 # ensures the calculation starts at the right point of the stress series
    W_pred = np.trapz(stress_pred[stress_start:], strain[stress_start:])
    W_data = data[4]

    loss_data_1 = nn.L1Loss(sig_pl_pred, sig_pl_data) + nn.L1Loss(W_pred, W_data)

    loss_physics = nn.L1Loss(stress_pred, stress_series_constit_eqn)

    loss_data_2 = nn.L1Loss(constit_eqn_coeffs, data[3])

    return loss_data_1 + loss_physics + loss_data_2

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F

In [ ]:
def Dynamic_Stress_PINN(nn.Module):
    
    def __init__(self, params):
        numparams = len(params)
        linear_in_dim = 1024 + numparams
        linear_out_dims = 5
        super(Dynamic_Stress_PINN, self).__init__()

        self.stress_ser_predictor = nn.Sequential


        self.constit_eqn_coeff_predictor = nn.Sequential(
            nn.Linear(linear_in_dim, 1024),
            nn.Linear(1024, 512),nn.ReLU(),
            nn.Linear(512, 256),nn.ReLU(),
            nn.Linear(256, 128),nn.ReLU(),
            nn.Linear(128, linear_out_dims)
        )


    def forward(self, x):
        feature_vector = x[0]
        property_vector = x[1]
        input_vec = torch.cat([feature_vector, property_vector])

        stress_ser = self.stress_series_predictor(input_vec) # MAKE SURE YOU KNOW IT KNOWS WHEN TO STOP... PROBABLY TAKEN CARE OF BY PADDED SET

        constit_eqn_coeffs = self.constit_eqn_coeff_predictor(input_vec)

        return stress_ser, constit_eqn_coeffs


# return featvec, paramvec, stress_series, constit_eqn_coeffs, W, sig_pl, strain


In [ ]:
def Dynamic_Stress_PINN(nn.Module):
    
    def __init__(self, params, series_input_dim=1, hidden_size=128, num_lstm_layers=4, lstm_output_dim=1):
        numparams = len(params)
        linear_in_dim = 1024 + numparams
        linear_out_dims = 5
        # self.series_in_dim = series_input_dim
        # self.hidden_size = hidden_size
        # self.num_lstm_layers = num_lstm_layers
        # self.lstm_output_dim = lstm_output_dim
        super(Dynamic_Stress_PINN, self).__init__()

        self.stress_ser_predictor = nn.LSTM(series_input_dim, hidden_size, num_lstm_layers, batch_first=True),
        self.lstm_linear = nn.Linear(hidden_size, lstm_output_dim)


        self.constit_eqn_coeff_predictor = nn.Sequential(
            nn.Linear(linear_in_dim, 1024),
            nn.Linear(1024, 512),nn.ReLU(),
            nn.Linear(512, 256),nn.ReLU(),
            nn.Linear(256, 128),nn.ReLU(),
            nn.Linear(128, linear_out_dims)
        )


    def forward(self, x):
        feature_vector = x[0]
        property_vector = x[1]
        input_vec = torch.cat([feature_vector, property_vector])

        stress_ser, _ = self.stress_series_predictor(input_vec) # MAKE SURE YOU KNOW IT KNOWS WHEN TO STOP... PROBABLY TAKEN CARE OF BY PADDED SET
        stress_ser = self.lstm_linear(stress_ser)

        constit_eqn_coeffs = self.constit_eqn_coeff_predictor(input_vec)

        return stress_ser, constit_eqn_coeffs


# return featvec, paramvec, stress_series, constit_eqn_coeffs, W, sig_pl, strain
